In [1]:
%matplotlib inline
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

import utils
import visualize
from visualize import display_images
import model as modellib
from config import Config
from model import log
import detector

import seaborn as sns
import pandas as pd

import imgaug as ia
from imgaug import augmenters as iaa
import imageio

import norfair
from norfair import Detection, Paths, Video

from typing import List

%matplotlib inline

# Root directory of the project
ROOT_DIR = os.path.abspath(".")
print(ROOT_DIR)
MODEL_DIR=os.path.join(ROOT_DIR, "models\logs")
print(MODEL_DIR)
COCO_MODEL_PATH=os.path.join(ROOT_DIR, "models\mask_rcnn_taco0100.h5")
print(COCO_MODEL_PATH)

C:\Users\blais\anaconda3\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper
Using TensorFlow backend.


C:\Users\blais\TACO-master\TACO-master\detector\models\logs
C:\Users\blais\TACO-master\TACO-master\detector
C:\Users\blais\TACO-master\TACO-master\detector\models\logs
C:\Users\blais\TACO-master\TACO-master\detector\models\mask_rcnn_taco0100.h5


In [2]:
import csv
import dataset

# Load class map - these tables map the original TACO classes to your desired class system
# and allow you to discard classes that you don't want to include.
class_map = {}
with open("./taco_config/map_10.csv") as csvfile:
    reader = csv.reader(csvfile)
    class_map = {row[0]:row[1] for row in reader}

# Load full dataset or a subset
TACO_DIR = "../data"
round = None # Split number: If None, loads full dataset else if int > 0 selects split no 
subset = "test" # Used only when round !=None, Options: ('train','val','test') to select respective subset
dataset = dataset.Taco()
taco = dataset.load_taco(TACO_DIR, round, subset, class_map=class_map, return_taco=True)

# Must call before using the dataset
dataset.prepare()

print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

creating index...
index created!
Number of images used: 1500
Class Count: 11
  0. BG                                                
  1. Bottle                                            
  2. Bottle cap                                        
  3. Can                                               
  4. Cigarette                                         
  5. Cup                                               
  6. Lid                                               
  7. Other                                             
  8. Plastic bag + wrapper                             
  9. Pop tab                                           
 10. Straw                                             


In [3]:
#Mask RCNN Configuration
class TacoTestConfig(Config):
    NAME = "taco"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 10
    NUM_CLASSES = dataset.num_classes
config = TacoTestConfig()

In [4]:
model=modellib.MaskRCNN(mode="inference", model_dir=TACO_DIR, config=config)
print(MODEL_DIR)
#inpath="./models/logs/mask_rcnn_taco_0100.h5"
#outpath = "C:/Users/blais/Internship/TACO-master/TACO-master/data"
#outpath="./models/logs/mask_rcnn_taco_0100.h5"
model_path="./models/logs/mask_rcnn_taco_0100.h5"







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
C:\Users\blais\TACO-master\TACO-master\detector\models\logs


In [5]:
#model.load_weights(weights_in_path=model_path, weights_out_path=model_path,
# by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox","mrcnn_mask"])
model.load_weights(weights_in_path=model_path, weights_out_path=model_path, by_name=True)

In [6]:
def norftrack(res):
    norftection: List[Detection]=[]
    for x in range (len(res['rois'])):
        box=np.array([[res['rois'][x][1], res['rois'][x][0]], [res['rois'][x][3], res['rois'][x][2]]])
        #box=np.array([[res['rois'][x][0], res['rois'][x][1]], [res['rois'][x][2], res['rois'][x][3]]])
        scores=np.array([res['scores'][x], res['scores'][x]])
        norftection.append(Detection(points=box, scores=scores))
    return norftection

In [7]:
def norftrack2(res):
    norftection: List[Detection]=[]
    for x in range(len(res['rois'])):
        yc=(res['rois'][x][0]+res['rois'][x][2])/2
        xc=(res['rois'][x][1]+res['rois'][x][3])/2
        
        centroid=np.array([xc, yc])
        scores=np.array([res['scores'][x]])
        norftection.append(Detection(points=centroid, scores=scores))
        
    return norftection
        

In [8]:
def euc_distance(detection, tracked_obj):
    #print(tracked_obj.estimate)
    return np.linalg.norm(detection.points-tracked_obj.estimate)

In [9]:
def key_distance(detection, tracked_obj):
    box1=np.concatenate([detection.points[0], detection.points[1]])
    box2=np.concatenate([tracked_obj.estimate[0], tracked_obj.estimate[1]])
    
    #print(box1)
    #print(box2)
    ya=max(box1[0], box2[0])
    xa=max(box1[1], box2[1])
    yb=max(box1[2], box2[2])
    xb=max(box1[3], box2[3])
    
    area=max(0, xb-xa+1)*max(0, yb-ya+1)
    
    aarea=(box1[2]-box1[0]+1)*(box1[3]-box1[1]+1)
    barea=(box2[2]-box2[0]+1)*(box2[3]-box2[1]+1)
    
    iou=area/float(aarea+barea-area)
    
    return 1/iou if iou else (10000)

In [10]:
def center(pnt):
    return[np.mean(np.array(pnt), axis=0)]

In [12]:
import cv2 as cv
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
# make a list to store all the frames
vid_array = []
# getting the video
vid2 = cv.VideoCapture("../data/vid2/sample_10s.mp4")  # the video i used is about a second long

#tracker=norfair.Tracker(distance_function=key_distance, distance_threshold=3.33)
tracker=norfair.Tracker(distance_function=euc_distance, distance_threshold=30)

path=Paths(center, attenuation=0.01)
# checks whether frames were extracted
success = 1

while success:
    # read the frame and add it to the frame list
    success, image = vid2.read()
    vid_array.append(image)
count=0
for img in vid_array:
    if count<(len(vid_array)-1):
        img=cv.cvtColor(img, cv.COLOR_BGR2RGB)
        vid_array[count]=img
    
    count+=1
# go through and detect all frames in the list  
counter=0
totalitems=[]
line=400


for vid_img in vid_array:
    if counter<(len(vid_array)-1):
        #vid_img=cv.cvtColor(vid_img, cv.COLOR_BGR2RGB)
        orimage=img_to_array(vid_img)
        results=model.detect([orimage], verbose=1)
        r=results[0]
        print(counter)
        #print(r['rois'])
        counter+=1
        #det=norftrack(r)
        det=norftrack2(r)
        tracked=tracker.update(detections=det)



        #norfair.draw_boxes(orimage, det)
        norfair.draw_points(orimage, det)

        #print(tracked)
        #print(tracked)
        itemsdet=0
        if (counter>=7):
            for item in tracked:
                #print(item.id)
                #print(item.estimate[0][1])
                if ((line+15)>item.estimate[0][1]>(line-15)):
                    print("item", item.id, "detected")
                    totalitems.append(item.id)
            totalitems=[*set(totalitems)]
            print(totalitems)

        #norfair.draw_tracked_boxes(orimage, tracked)
        norfair.draw_tracked_objects(orimage, tracked, color=(0, 225, 0), id_size=3, id_thickness=2)

        frame=path.draw(vid_img, tracked)
        orimage=cv.line(orimage, (0,line), (2000, line), (255, 0, 0), 6)

        visualize.display_instances(orimage, r['rois'], r['masks'], r['class_ids'], dataset.class_names, r['scores'])
        plt.close()  # close the figure after displaying it to free up memory
print(len(totalitems))

Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  float32


KeyboardInterrupt: 